# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

library를 사용할 때 사용자는 애플리케이션 코드의 흐름을 직접 제어해야 한다.
개발 시 필요한 기능이 있을 경우 능동적으로 라이브러리를 호출하여 사용하거나 기존에 구성된 함수나 코드를 가져다 써야 한다. 반면 프레임워크는 애플리케이션의 코드가 프레임워크에 의해 사용된다. 애플리케이션 코드는 프레임워크가 짜 놓은 틀에서 수동적으로 동작하기 때문에 제어의 흐름은 프레임워크가 가지고 있고 사용자가 그 안에 필요한 코드를 작성하게 된다. 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

딥러닝과 머신러닝의 관계는 딥러닝은 머신러닝에 포함된 영역이다. 

딥러닝의 특징은 여러 층을 가진 인공신경망을 시용하여 머신러닝 학습을 수행하는 것으로 심층학습망이라고도 불린다. 인간의 뇌가 가지고 있는 생물학적 특성 중 뉴런의 연결 구조를 본떠서 만든 네트워크 구조이다. 인간의 뇌는 수많은 뉴런이 모여 만든 신호의 흐름을 기반으로 다양한 사고를 할 수 있다. 인간의 뇌를 본떠서 만든 딥러닝도 뉴런이라고 불리는 퍼셉트론 알고리즘을 기반하여 다층 퍼셉트론의 조합으로 다양한 스스로 학습이 가능하다. 머신러닝은 컴퓨터가 스스로 학습할 수 있도록 도와주는 알고리즘이나 기술을 개발하는 분야를 말한다. 양질의 데이터를 이용하여 어떠한 알고리즘, 기술을 사용해서 데이터를 분석, 분석 결과를 스스로 학습한 후, 이를 기반으로 어떠한 판단이나 예측을 하는 것을 의미한다. 

딥러닝과 머신러닝의 가장 큰 차이점은 바로 기계의 자가 학습 여부라고 할 수 있다.
머신러닝은 사람이 개입하여 컴퓨터에 특정 패턴 추출 방법을 지시하지만, 딥러닝은 스스로 데이터를 기반으로 패턴을 파악하여 결과를 도출해낼 수 있다.


### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms ##필요한 패키지 import 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")   ## cpu에서 gpu로 변환

In [ ]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5               ## 학습률 , 배치 사이즈, number 갯수, 반복횟수 설정

In [ ]:

train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)        ##MNIST 데이터중 train SET 설정

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)        ##MNIST 데이터중 test SET 설정

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:

class ConvNet(nn.Module):       ##컨볼루션 모델 class
  def __init__(self):  
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10)  ##모델의 층 설정


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)  ## 모델이 학습하는 과정 설정

In [ ]:

model = ConvNet().to(device) ##모델 불러오기
criterion = nn.CrossEntropyLoss().to(device) ##모델 loss 설정
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) ##모델 옵티마이저 설정

In [ ]:

for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader)  ## 모델 훈련 파라미터 설정
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:

model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item()  ## 데이터와 타겟의 맞는 비율을 계산
        
    print('Test Accuracy: ', 100.*correct/total, '%') ## 최종 acc 출력
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.